# 신경망 모델 구성하기

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'device : {device}')

device : cuda


## 클래스 정의하기

In [11]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )
    
    def forward(self, x):
        x = self.flatten(x) # size = (1,784)
        logits = self.linear_relu_stack(x) # size = (1, 10)
        return logits

In [12]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [13]:
X = torch.rand(1,28,28, device=device)
logits = model(X)
pred_prob = nn.Softmax(dim=1)(logits)
y_pred = pred_prob.argmax(1)    # 1은 차원을 의미. 즉, 열 단위로 뽑기
print(f'Predicted class : {y_pred}')

Predicted class : tensor([0], device='cuda:0')


## 모델 계층

In [15]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten

In [16]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear

In [17]:
layer1 = nn.Linear(28*28, 20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU

In [19]:
print(f'Before ReLU: {hidden1}')
hidden1 = nn.ReLU()(hidden1)
print(f'After ReLU: {hidden1}')

Before ReLU: tensor([[ 0.1693,  0.2132, -0.1209,  0.0852,  0.2500, -0.1913,  0.0761, -0.5355,
         -0.2086,  0.1902,  0.0116, -0.1390, -0.1305, -0.0987,  0.1810, -0.4464,
          0.8204,  0.2065, -0.3069,  0.0533],
        [ 0.3761,  0.3660,  0.1378, -0.2055,  0.4471, -0.2808,  0.1534, -0.4979,
         -0.1249,  0.1870,  0.0505, -0.4115, -0.0234,  0.1878,  0.1517, -0.5401,
          0.4455, -0.0353, -0.2260, -0.0450],
        [ 0.5331,  0.1349, -0.1284, -0.1350,  0.1201, -0.5375,  0.0086, -0.4950,
         -0.2473, -0.1536,  0.0844, -0.2158,  0.0610,  0.0303,  0.0306, -0.3342,
          0.4170,  0.0331,  0.1891,  0.0157]], grad_fn=<AddmmBackward0>)
After ReLU: tensor([[0.1693, 0.2132, 0.0000, 0.0852, 0.2500, 0.0000, 0.0761, 0.0000, 0.0000,
         0.1902, 0.0116, 0.0000, 0.0000, 0.0000, 0.1810, 0.0000, 0.8204, 0.2065,
         0.0000, 0.0533],
        [0.3761, 0.3660, 0.1378, 0.0000, 0.4471, 0.0000, 0.1534, 0.0000, 0.0000,
         0.1870, 0.0505, 0.0000, 0.0000, 0.1878, 0.1517

### nn.Sequential

In [21]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)

input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

### nn.Softmax

In [22]:
softmax = nn.Softmax(dim=1)
pred_prob = softmax(logits)

### 모델 매개변수

In [24]:
print(f'Model structure: {model} \n\n')

for name, param in model.named_parameters():
    print(f'Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n')

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0254, -0.0063, -0.0352,  ..., -0.0254,  0.0101, -0.0132],
        [ 0.0074,  0.0342, -0.0051,  ...,  0.0239, -0.0135,  0.0202]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0073, 0.0058], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0001, -0.0165, -0.0357,  ...,  0.0059, -0.0012, -0.0112],
        [ 0.0297,  0.0191,  0.0145,  ...,  0.0323,  0.0403, -0.0398]],
       device='cuda:0', grad_fn=<Sli